In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
pip install python-terrier==0.10.0 nltk scikit-learn lightgbm xgboost fastrank

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# !pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR

In [4]:
import pandas as pd
import json
import pyterrier as pt
import random

In [5]:
#UNCOMMENT THE FOLLOWING LINE TO USE EITHER THE TVR DATASET OR THE QVH DATASET

dataset_choice = "TVR"
# dataset_choice = "QVH"

In [6]:
# Paths to JSONL files on Colab
# if dataset_choice == "TVR":
#    jsonl_train_path = '/content/drive/MyDrive/IR/text_data/tvr_train_release.jsonl'
#    jsonl_val_path = '/content/drive/MyDrive/IR/text_data/tvr_val_release.jsonl'
#    subs_path = '/content/drive/MyDrive/IR/text_data/tvqa_preprocessed_subtitles.jsonl'

# elif dataset_choice == "QVH":
#    jsonl_train_path = "/content/drive/MyDrive/IR/text_data_QVH/highlight_train_release.jsonl"
#    subs_path = "/content/drive/MyDrive/IR/text_data_QVH/subs_train.jsonl"

In [7]:
# Paths to JSONL files
if dataset_choice == "TVR":
    jsonl_train_path = '/kaggle/input/tvretrieval/tvr_train_release.jsonl'
    jsonl_val_path = '/kaggle/input/tvretrieval/tvr_val_release.jsonl'
    subs_path = '/kaggle/input/tvretrieval/tvqa_preprocessed_subtitles.jsonl'

elif dataset_choice == "QVH":
    jsonl_train_path = "text_data_QVH/highlight_train_release.jsonl"
    subs_path = "text_data_QVH/subs_train.jsonl"

In [8]:
# Load subtitles into a dictionary for quick access
subtitles_dict = {}
if dataset_choice == "TVR":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            subtitles_dict[sub_data['vid_name']] = sub_data['sub']
elif dataset_choice == "QVH":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            triple = sub_data['vid'].split("_")
            name = triple[0:-2]
            #turn the list name into a string
            name = "".join(name)
            if name not in subtitles_dict:
                subtitles_dict[name] = [(float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query'])]
            else:
                subtitles_dict[name].append((float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query']))

In [9]:
# Function to find matching subtitles in TVR case
def find_matching_subtitles(vid_name, ts_range, subtitles_dict):
    matching_subs = []
    if vid_name in subtitles_dict:
        for subtitle in subtitles_dict[vid_name]:
            if (ts_range[0] <= subtitle['start'] <= ts_range[1]) or (ts_range[0] <= subtitle['end'] <= ts_range[1]) or (subtitle['start'] <= ts_range[0] and subtitle['end'] >= ts_range[1]):
                matching_subs.append(subtitle['text'])
    return matching_subs

In [10]:
def parse_jsonl_TVR(jsonl_path, split_type):
    # Initialize empty lists for your data
    queries_data = []
    documents_data = []
    query_rankings_data = []

    with open(jsonl_path, 'r') as file:
        for idx, line in enumerate(file):
            data = json.loads(line)
            # drop non text-based queries
            if data['type'] not in ['t']:
                continue

            # Find matching subtitles
            matching_subs = find_matching_subtitles(data['vid_name'], data['ts'], subtitles_dict)

            if matching_subs == []:
                continue

            # Extract data for the Query Set DataFrame
            queries_data.append({'qid': str(data['desc_id']), 'query': data['desc']})

            # Extract data for the Documents Set DataFrame, including matching subtitles
            documents_data.append({'docno': split_type + str(idx), 'vid_name': data['vid_name'], 'ts': data['ts'],
                                'duration': data['duration'], 'type': data['type'], 'text': "".join(matching_subs)})

            # Extract data for the Query Rankings DataFrame
            query_rankings_data.append({'qid': str(data["desc_id"]), 'query': data['desc'], 'docno': split_type + str(idx), 'rank': 1, 'score': 1.0})

    return queries_data, documents_data, query_rankings_data

def parse_jsonl_QVH(jsonl_path):
    queries_data = []
    documents_data = []
    query_rankings_data = []
    with open(jsonl_path, 'r') as file:
        for idx,line in enumerate(file):

            # Load the JSON object from the line
            data = json.loads(line)

            triple = data["vid"].split("_")
            document_name = triple[0:-2]
            document_name = "".join(document_name)
            start_time = float(triple[-2])
            end_time = float(triple[-1])

            if document_name not in subtitles_dict:
                #print("Document not found in subtitles: ", document_name)
                continue

            momentaneus_rank =[]
            count = 0
            for id,relevant_window in enumerate(data["relevant_windows"]):
                ts = [start_time+relevant_window[0], start_time+relevant_window[1]]
                subs = [sub for sub in subtitles_dict[document_name] if sub[0] <= ts[1] and ts[0] <= sub[1]]
                if len(subs) == 0:
                    #print("No subtitles found for ", document_name, " at time ", ts)
                    continue
                count += 1
                documents_data.append({"docno" : str(idx) +"_"+str(ts[0]) + "_" + str(ts[1]), "vid_name" : document_name, "ts": ts, "duration": data["duration"], "text": "".join([sub[2] for sub in subs])})
                scores = [data["saliency_scores"][i]  for i,clip_id in enumerate(data["relevant_clip_ids"]) if clip_id*2 >= relevant_window[0] and clip_id*2 <= relevant_window[1]]
                if len(scores) == 0:
                    #print("No scores found for ", document_name, " at time ", ts)
                    continue
                #each entry of scores is a triple of integers. Create a variable score which is the average of all the scores
                score = 0 if len(scores) ==0 else sum(sum(scores[i]) for i in range(len(scores)))/(3*len(scores))

                momentaneus_rank.append({"qid" : str(data["qid"]), "query": data["query"] , "docno" : str(idx) +"_"+str(ts[0]) + "_" + str(ts[1]), "score": score, "rank":1})

            if count == 0:
                #print("No relevant windows found for ", document_name)
                continue
            #adjust the rank of the momentaneus_rank based on the score
            momentaneus_rank = sorted(momentaneus_rank, key=lambda x: x["score"], reverse=True)
            for i in range(len(momentaneus_rank)):
                momentaneus_rank[i]["rank"] = i+1
            queries_data.append({"qid" : str(data["qid"]), "query": data["query"]})
            query_rankings_data.extend(momentaneus_rank)

    return queries_data, documents_data, query_rankings_data

In [11]:

if dataset_choice == "TVR":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_TVR(jsonl_train_path, "t")
    queries_data_val, documents_data_val, query_rankings_data_val = parse_jsonl_TVR(jsonl_val_path, "v")
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]

elif dataset_choice == "QVH":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_QVH(jsonl_train_path)
    #have to create a val set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_val = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_val]
    queries_data_val = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_val]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_val]
    documents_data_val = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_val]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_val]
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]



In [12]:
# Create DataFrames for the Query Set, Documents Set, and Query Rankings
queries_train_df = pd.DataFrame(queries_data_train)
documents_train_df = pd.DataFrame(documents_data_train)

queries_val_df = pd.DataFrame(queries_data_val)
documents_val_df = pd.DataFrame(documents_data_val)

queries_test_df = pd.DataFrame(queries_data_test)
documents_test_df = pd.DataFrame(documents_data_test)

q_rels = pd.concat([pd.DataFrame(query_rankings_data_train), pd.DataFrame(query_rankings_data_val), pd.DataFrame(query_rankings_data_test)]).reset_index(drop=True)

#print length of the dataframes
print("Train set:")
print("Queries: ", len(queries_train_df))
print("Documents: ", len(documents_train_df))

print("Val set:")
print("Queries: ", len(queries_val_df))
print("Documents: ", len(documents_val_df))

print("Test set:")
print("Queries: ", len(queries_test_df))
print("Documents: ", len(documents_test_df))

print("Query Rankings: ", len(q_rels))



Train set:
Queries:  7110
Documents:  7110
Val set:
Queries:  957
Documents:  957
Test set:
Queries:  790
Documents:  790
Query Rankings:  8857


### First Stage Retrieval [TODO: BOX]
The following part of the code will define three different first stage retrieval pipelines as an input for the trained model.

In [13]:
!ls -la /usr/lib/jvm/java-11-openjdk-amd64
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

if not pt.started():
    pt.init()

total 32
drwxr-xr-x  7 root root 4096 Feb 22 12:31 .
drwxr-xr-x  3 root root 4096 Feb 22 12:31 ..
drwxr-xr-x  2 root root 4096 Feb 22 12:31 bin
drwxr-xr-x  4 root root 4096 Feb 22 12:31 conf
lrwxrwxrwx  1 root root   42 Oct 19 07:55 docs -> ../../../share/doc/openjdk-11-jre-headless
drwxr-xr-x 73 root root 4096 Feb 22 12:31 legal
drwxr-xr-x  6 root root 4096 Feb 22 12:31 lib
drwxr-xr-x  4 root root 4096 Feb 22 12:31 man
-rw-r--r--  1 root root 1248 Oct 19 07:55 release
terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [14]:
# Create an index
from pathlib import Path

indexer = pt.IterDictIndexer(
    "./index_path/",
    meta={
        "docno": 64,
        "vid_name": 64,
        "text": 131072,
    },
    stemmer="porter",
    stopwords="terrier",
    overwrite=True,
    # type=pt.index.IndexingType.MEMORY,
)

In [15]:
joint_documents_set_df = pd.concat([documents_train_df, documents_val_df, documents_test_df])

print("Length: ", len(joint_documents_set_df))

Length:  8857


In [16]:
indexed = indexer.index(
    joint_documents_set_df.to_dict(orient="records")
)

12:25:52.092 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (t24499) - further warnings are suppressed
12:25:57.412 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


In [17]:
#These are some examples, not necessarily the best ones. Experiment with different models

# Initialize BatchRetrieve with the created index and specify BM25 as the weighting model
first_stage_bm25 = pt.BatchRetrieve(
    indexed,
    wmodel="BM25",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify LemurTF_IDF as the weighting model
first_stage_lemurtfidf = pt.BatchRetrieve(
    indexed,
    wmodel="LemurTF_IDF",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify Hiemstra_LM as the weighting model
first_stage_hiemstra_lm = pt.BatchRetrieve(
    indexed,
    wmodel="Hiemstra_LM",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

first_stage_dfic = pt.BatchRetrieve(
    indexed,
    wmodel="DFIC",
    num_results=100,
    metadata=['docno', 'vid_name', 'text']
)

first_stage_ldg = pt.BatchRetrieve(
    indexed,
    wmodel="LGD",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

first_stage_in_exp_b2 = pt.BatchRetrieve(
    indexed,
    wmodel="In_expB2",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

# Computing feature
The weighting model can be use in [pt.weighting_model.package: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html)

In [18]:
#We create features for the second stage using the first stage retrievers

#TF-IDF based features
lemur_tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="LemurTF_IDF")
bm25_retriever = pt.BatchRetrieve(indexed, wmodel="BM25")
tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="TF_IDF")

#Language model based features
hiem_retriever = pt.BatchRetrieve(indexed, wmodel="Hiemstra_LM")
dirichlet_retriever = pt.BatchRetrieve(indexed, wmodel="DirichletLM")

#Divergence from randomness based features
pl2_retriever = pt.BatchRetrieve(indexed, wmodel="PL2")
dph_retriever = pt.BatchRetrieve(indexed, wmodel="DPH")
dlh_retriever = pt.BatchRetrieve(indexed, wmodel="DLH")

#Can add more!
coordinate_match_retriever = pt.BatchRetrieve(indexed, wmodel="CoordinateMatch")
js_kls_retrieveer = pt.BatchRetrieve(indexed, wmodel="Js_KLs")


In [19]:
# PIPELINES WITH FSR AND FEATURES

#Can test many possibilities, i just put some examples here

bm25_pipeline = ~first_stage_bm25 >> (
   tf_idf_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer ** bm25_retriever
)


lemurtf_idf_pipeline = ~first_stage_lemurtfidf >> (
    pl2_retriever ** dph_retriever ** hiem_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer
)

hiem_lm_pipeline = ~hiem_retriever >> (
    pl2_retriever ** dph_retriever ** tf_idf_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer
)

#NOTE: i believe we should not use the same first stage retriever for features AND for the first stage retrieval

In [20]:
# Prepare the queries for the pipeline, remove special characters and extra spaces
prepared_trainqueries = queries_train_df
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels = pd.DataFrame(query_rankings_data_train)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels['label'] = prepared_train_qrels['score']

prepared_train_qrels['label'] = prepared_train_qrels['label'].astype(int)

prepared_val_qrels = pd.DataFrame(query_rankings_data_val)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_val_qrels['label'] = prepared_val_qrels['score']

prepared_val_qrels['label'] = prepared_val_qrels['label'].astype(int)

prepared_test_qrels = pd.DataFrame(query_rankings_data_test)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_test_qrels['label'] = prepared_test_qrels['score']

prepared_test_qrels['label'] = prepared_test_qrels['label'].astype(int)

prepared_qrels = q_rels
prepared_qrels['query'] = prepared_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_qrels['label'] = prepared_qrels['score']

prepared_qrels['label'] = prepared_qrels['label'].astype(int)

prepared_valqueries = queries_val_df.reset_index()
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)

prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

#test set
prepared_testqueries = queries_test_df.reset_index()
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()


Storage format: .csv or trec

trec(https://pyterrier.readthedocs.io/en/latest/io.html): The pt io format, but it doesn't contain feature.

In [21]:
from sklearn.svm import SVR
import xgboost as xgb
import fastrank

index = pt.IndexFactory.of("./index_path")

fsr_pipelines = [
    #{
    #    'pipe': lemurtf_idf_pipeline,
    #    'name': 'LemurTF_IDF'
    #},
    #{
    #    'pipe': bm25_pipeline,
    #    'name': 'BM25'
    #},
    #{
    #    'pipe': hiem_lm_pipeline,
    #    'name': 'Hiemstra LM'
    #}
]

learned_models = [
    # {
    #     'model': SVR(),
    #     'form': 'reg',
    #     'name': 'SVR',
    # },
    {
        'model': xgb.XGBRanker(tree_method="hist", objective="rank:ndcg"),
        'form': 'ltr',
        'name': 'XGBoost (NDCG)',
    },
    #{
    #    'model': xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:pairwise", lambdarank_pair_method="topk"),
    #    'form': 'ltr',
    #    'name': 'XGBoost (Pariwise)'
    #}
    # {
    #     'model': xgb.XGBRanker(tree_method="hist", objective="rank:map"),
    #     'form': 'ltr',
    #     'name': 'XGBoost (MAP)',
    # },
    #{
    #    'model': fastrank.TrainRequest.coordinate_ascent(),
    #    'form': 'fastrank',
    #    'name': 'FastRank Coordinate Ascent',
    #},
    #{
    #    'model': fastrank.TrainRequest.random_forest(),
    #    'form': 'fastrank',
    #    'name': 'FastRank Random Forest',
    #}
  ]

In [22]:
from pyterrier.measures import *

trained_models = [first_stage_bm25, first_stage_lemurtfidf, first_stage_hiemstra_lm, first_stage_dfic,  first_stage_ldg, first_stage_in_exp_b2]
names = ['BM25', 'LemurTF_IDF', 'Hiemstra LM', 'DFIC', 'LGD', 'In Exp  B2']
eval_metrics = [
    nDCG @ 1, nDCG @ 3, nDCG @ 5, nDCG @ 10, nDCG @ 30,
    RR @ 1,   RR @ 3,   RR @ 5,   RR @ 10, RR @ 30,
    MAP, NumRelRet, Judged @ 1,Judged @ 3, Judged @ 5
]

for fsr in fsr_pipelines:
    for model in learned_models:
        names.append(f"{fsr['name']} >> {model['name']}")
        print(names[-1])
        if 'form' in model:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'], form=model['form'])
        else:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'])
        pipe.fit(
            prepared_trainqueries,
            prepared_train_qrels,
            prepared_valqueries,
            prepared_val_qrels
        )
        trained_models.append(pipe)

In [23]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_testqueries,
    prepared_test_qrels,
    names=names,
    eval_metrics=eval_metrics,
)


,name,nDCG@1,nDCG@3,nDCG@5,nDCG@10,nDCG@30,RR@1,RR@3,RR@5,RR@10,RR@30,AP,NumRet(rel=1),Judged@1,Judged@3,Judged@5
0,BM25,0.193671,0.239239,0.251602,0.269062,0.289388,0.193671,0.228270,0.235042,0.242186,0.247235,0.249689,451.0,0.193671,0.090295,0.060253
1,LemurTF_IDF,0.175949,0.221186,0.235462,0.249067,0.270555,0.175949,0.210127,0.218165,0.223643,0.228794,0.230988,429.0,0.175949,0.084388,0.057468
2,Hiemstra LM,0.183544,0.226084,0.237523,0.253003,0.271873,0.181013,0.213924,0.220506,0.226934,0.231605,0.235654,430.0,0.181013,0.084810,0.056709
3,DFIC,0.179747,0.217057,0.229476,0.251819,0.269983,0.179747,0.208439,0.215274,0.224529,0.228980,0.231538,439.0,0.179747,0.080591,0.054430
4,LGD,0.173418,0.214526,0.230538,0.247698,0.268238,0.173418,0.204641,0.213439,0.220754,0.225674,0.228150,437.0,0.173418,0.081013,0.056456
5,In Exp B2,0.192405,0.244966,0.257873,0.273979,0.295754,0.192405,0.232068,0.239156,0.245546,0.250858,0.253325,463.0,0.192405,0.094093,0.062785


In [24]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_valqueries,
    prepared_val_qrels,
    names=names,
    eval_metrics=eval_metrics,
)

,name,nDCG@1,nDCG@3,nDCG@5,nDCG@10,nDCG@30,RR@1,RR@3,RR@5,RR@10,RR@30,AP,NumRet(rel=1),Judged@1,Judged@3,Judged@5
0,BM25,0.196447,0.246292,0.264185,0.288406,0.309813,0.197492,0.234413,0.243974,0.254232,0.259449,0.261373,573.0,0.197492,0.093347,0.064577
1,LemurTF_IDF,0.167189,0.227532,0.243763,0.258096,0.281515,0.167189,0.211947,0.221351,0.227320,0.233054,0.235721,548.0,0.167189,0.089516,0.062069
2,Hiemstra LM,0.179728,0.229138,0.244560,0.262100,0.285058,0.179728,0.215778,0.224608,0.231965,0.237554,0.240082,538.0,0.179728,0.087774,0.060397
3,DFIC,0.174504,0.223850,0.238152,0.257576,0.286947,0.174504,0.212295,0.220080,0.228004,0.235334,0.237360,556.0,0.174504,0.085684,0.058098
4,LGD,0.179728,0.226189,0.239857,0.258974,0.288781,0.179728,0.215430,0.222745,0.230653,0.238008,0.240123,551.0,0.179728,0.085684,0.057889
5,In Exp B2,0.187043,0.255546,0.269169,0.290187,0.312461,0.188088,0.238419,0.246151,0.255024,0.260360,0.262796,594.0,0.188088,0.100662,0.067294


In [25]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_trainqueries[:1000],
    prepared_train_qrels[:1000],
    names=names,
    eval_metrics=eval_metrics,
)

,name,nDCG@1,nDCG@3,nDCG@5,nDCG@10,nDCG@30,RR@1,RR@3,RR@5,RR@10,RR@30,AP,NumRet(rel=1),Judged@1,Judged@3,Judged@5
0,BM25,0.193,0.248249,0.264344,0.282649,0.308248,0.193,0.234667,0.243817,0.251682,0.257874,0.260094,611.0,0.193,0.095333,0.0652
1,LemurTF_IDF,0.178,0.225439,0.241790,0.259810,0.283194,0.178,0.213500,0.222950,0.230040,0.235929,0.238477,591.0,0.178,0.086000,0.0600
2,Hiemstra LM,0.170,0.225249,0.243023,0.262364,0.282897,0.171,0.212000,0.221950,0.229875,0.234765,0.236672,571.0,0.171,0.088000,0.0614
3,DFIC,0.173,0.227094,0.245211,0.262290,0.287101,0.173,0.214000,0.223750,0.230844,0.236745,0.239239,598.0,0.173,0.089000,0.0620
4,LGD,0.175,0.228285,0.243175,0.261488,0.287122,0.175,0.214500,0.222800,0.230433,0.236900,0.239096,589.0,0.175,0.089667,0.0610
5,In Exp B2,0.204,0.256249,0.276344,0.294168,0.318248,0.205,0.244000,0.255350,0.262834,0.268601,0.270538,625.0,0.205,0.097000,0.0682
